In [ ]:
# For interactive matplotlib
%matplotlib widget

# Import the N_Queens_Game class from the mobile_game module
from mobile_game import N_Queens_Game

# Create an instance of N_Queens_Game
n_queens_game = N_Queens_Game()

# Setup the game interface
n_queens_game.setup()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Styling the button to look like a link or label
button_style = """
<style>
.button-style {
    background-color: transparent;
    color: black;
    border: none;
    text-decoration: underline;
    font-size: 12px;
}
</style>
"""

button = widgets.Button(description='AI', layout=widgets.Layout(width='auto'))
button.add_class("button-style")  # Applying CSS class

# Display button with style
display(widgets.HTML(value=button_style), button)

# Define button click action
def on_button_click(b):
    with output_area:
        clear_output(wait=True)
        print("Displaying new configuration setup...")

button.on_click(on_button_click)

output_area = widgets.Output()
display(output_area)

In [4]:
import IPython

IPython.display.display(IPython.display.Audio("./click.mp3", autoplay=True))
